In [65]:
from PIL import Image
from numpy import*
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
import imageio
import glob
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

In [66]:
def get_image():
    import imageio
    sequence_folder_0 = glob.glob('./cv_img_len/*.*')
    array_image=[]
    binary=[]
    for path in sequence_folder_0:
        im = imageio.imread(path)
        array = np.zeros((60,100,3))
        temp=im
        array[:temp.shape[0],:temp.shape[1],:temp.shape[2]] = temp
        array_image.append(array)
        binary.append([1.,0.])
     
    sequence_folder_1 = glob.glob('./cv_img_xuong/*.*')
    for path in sequence_folder_1:
        im = imageio.imread(path)
        array = np.zeros((60,100,3))
        temp=im
        array[:temp.shape[0],:temp.shape[1],:temp.shape[2]] = temp
        array_image.append(array)
        binary.append([0.,1.])
        
    return np.array(array_image),binary
    


In [67]:
array_image,binary=get_image()

In [68]:
array_image.shape

(495, 60, 100, 3)

In [69]:
binary = np.asarray(binary)
binary[0]

array([1., 0.])

In [70]:
X_train, X_val, y_train, y_val = train_test_split(array_image, binary, test_size=0.1, random_state=20)

In [87]:
X_train = X_train.reshape(X_train.shape[0], 60, 100, 3)
X_val = X_val.reshape(X_val.shape[0], 60, 100, 3)
# 4. One hot encoding label (Y)
Y_train = y_train
Y_val = y_val
print('Dữ liệu y ban đầu ', y_train[0])
print('Dữ liệu y sau one-hot encoding ',Y_train[0])


Dữ liệu y ban đầu  [1. 0.]
Dữ liệu y sau one-hot encoding  [1. 0.]


In [92]:
model = Sequential()
model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(60, 100, 3)))

# convolutional layer
model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(2, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 60, 100, 50)       1400      
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 60, 100, 75)       33825     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 30, 50, 75)        0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 30, 50, 75)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 30, 50, 125)       84500     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 15, 25, 125)       0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 15, 25, 125)     

In [93]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10,
            verbose=1,batch_size=3,callbacks=[es])

Epoch 1/10
149/149 [==============================] - 68s 458ms/step - loss: 0.5768 - accuracy: 0.7236 - val_loss: 0.2636 - val_accuracy: 0.9600
Epoch 2/10
149/149 [==============================] - 69s 460ms/step - loss: 0.3023 - accuracy: 0.9056 - val_loss: 0.2391 - val_accuracy: 0.9600
Epoch 3/10
149/149 [==============================] - 71s 473ms/step - loss: 0.2702 - accuracy: 0.9169 - val_loss: 0.2464 - val_accuracy: 0.9600
Epoch 00003: early stopping


In [94]:
from keras.models import load_model
model.save('image_binary.ctk')

INFO:tensorflow:Assets written to: image_binary.ctk/assets


In [95]:
y_sample = model.predict(X_val)
y_sample

array([[1.36548564e-01, 8.63451481e-01],
       [9.95858014e-01, 4.14203433e-03],
       [6.94594710e-05, 9.99930501e-01],
       [6.52011065e-03, 9.93479848e-01],
       [9.04357433e-01, 9.56425443e-02],
       [9.71152246e-01, 2.88477596e-02],
       [9.14607406e-01, 8.53926167e-02],
       [5.49416382e-06, 9.99994516e-01],
       [8.59090686e-01, 1.40909329e-01],
       [4.79732640e-04, 9.99520302e-01],
       [9.21016991e-01, 7.89829642e-02],
       [7.90465295e-01, 2.09534749e-01],
       [9.29182187e-13, 1.00000000e+00],
       [9.78444040e-01, 2.15559155e-02],
       [9.16034579e-01, 8.39653984e-02],
       [9.66499209e-01, 3.35008018e-02],
       [9.45028603e-01, 5.49714044e-02],
       [9.83122170e-01, 1.68778598e-02],
       [2.03769580e-02, 9.79623020e-01],
       [1.83790410e-03, 9.98162091e-01],
       [2.03889882e-04, 9.99796093e-01],
       [6.03515562e-03, 9.93964851e-01],
       [7.73806940e-04, 9.99226213e-01],
       [9.86651540e-01, 1.33484378e-02],
       [8.779261